In [ ]:
# Funciones de detección de adicionales. Cada una genera 1 columna
def tiene_SUM(df):
  df['tiene_SUM'] = df.description.str.contains('\bsum\b', regex = True)
  return df
def tiene_pileta(df):
  df['tiene_pileta'] = df.description.str.contains('pileta', regex = True)
  return df
def tiene_laundry(df):
  df['tiene_laundry'] = df.description.str.contains('laundry', regex = True)
  return df
def tiene_gimnasio(df):
  df['tiene_gimnasio'] = df.description.str.contains('gimnasio', regex = True)
  return df
def tiene_cochera(df):
  df['tiene_cochera'] = df.description.str.contains('cochera', regex = True)
  return df

# columnas generadas: 'tiene_SUM', 'tiene_pileta', 'tiene_laundry', 'tiene_gimnasio', 'tiene_cochera'

In [ ]:
# Detección de posible pozo. Puede ser perfeccionada
def posible_pozo(df):
  df['posible_pozo'] = df.description.str.contains('\bpozo\b', regex = True)
  return df
# columnas generadas: 'posible_pozo'

In [ ]:
#Regex más sofisticada en R con problemas para correr en python 
#Esta regex busca digitos precedidos por "antigüedad", opcionalmente con ":" entre antiguedad y los digitos, seguidos de "años"
#'(?<=((((antigüedad|antiguedad)(\\sde)?))(:|\\.|;)?(\\s){0,2}))\\b[1-9](?=\\s{0,2}años)'

#Esta es una función muy simple de extracción de años, extrae 1 o 2 caracteres que esten seguidos por string "años"
#Se propone no correr a menos que revoquemos la decisión en base a EDA
#def antiguedad_detectada(df):
#  df['antiguedad_detectada'] = df.description.str.findall(r'[0-9]+\.*[0-9]* *años')
#  return df

In [ ]:
# Detección de inmuebles a estrenar en texto libre
def deteccion_a_estrenar(df):
  df['a_estrenar'] = df.description.str.contains('\ba estrenar\b', regex = True)
  return df

### Creación de nuevas features a partir de la descripción de las propiedades

Creo una columna con el título y la descripción de las propiedades como para extraer algunas features adicionales

In [ ]:
data['texto'] = data['title_cleaned'] + " " + data['description_cleaned']

Busco propiedades con cochera. Armo una expresión regular para no incluir a aquellos que quieren venderla por separado

pattern_cochera = r'(?<!s\/)(?<!sin )(\d|con|la|las|c\/)?(cochera\.?)(?!s?\s?desde)(?!s?\s?\w*?\s?(op|cort|\d))( fija)?'

x = data['texto'].str.extract(pattern_cochera)

x.bfill(axis=1, inplace=True)
x = x.iloc[:,0]
x.loc[~x.isnull()] = True  # not nan
x.loc[x.isnull()] = False   # nan

data['has_cochera'] = x

Creo una función para extraer algunos amenities y características que pueden ser relevantes para calcular el precio de una propiedad.

In [ ]:
def apply_regex(columna, value):
  pattern_sin = r"((sin|no tiene|no contiene|no hay|no es|no esta|no incluye|no se permite|no se permiten|poca|nula)\s+("+ value +"))"
  pattern_ok = r"("+ value +")"
  # return Falso cuando no hay ninguno de los lugares
  if re.search(pattern_sin, columna, re.M|re.I) is not None:
    return False
  elif re.search(pattern_ok, columna, re.M|re.I) is not None:
    return True
  else:
    return False

creo las columnas adicionales buscando los términos en la columna "texto"

In [ ]:
amenities = ['patio', 'jardin', 'balcon', 'terraza', 'parrilla', 'sum', 'pileta|piscina', 'luminoso|luminosidad|mucha luz', 'laundry|lavadero|lavarropas', 'baulera', 'gimnasio|gym', 'seguridad|vigilancia', 'vestidor', 'a estrenar', 
             'pool|ping pong|metegol|microcine', 'portero|porteria|encargado', 'jacuzzi|sauna|solarium|yacuzzi', 'apto profesional', 'amenities', 'por escalera', 'a reciclar|para reciclar', 'categoria|primera linea', 'reciclado|reciclada',
             'en pozo|entrega en|de pozo|emprendimiento', 'tiro balanceado|losa radiante'] 

for i in amenities:
  col_name = "has_"+ i.replace(' ', '_').split("|")[0]
  data[col_name] = data['texto'].apply(lambda x: apply_regex(x, i))

In [ ]:
filter_col = [col for col in data if col.startswith('has')]

for col in filter_col:
  print(col, data[col].mean())

### Distancia al subte

Cargo un csv con las coordenadas geográficas de las estaciones de subte de CABA

In [4]:
import pandas as pd
url='https://drive.google.com/uc?id=1oxCEj_enxBVjBsDuvuy5w4HF02HasCWl'
subtes = pd.read_csv(url)
subtes.sample()

,long,lat,id,estacion,linea
6,-58.397924,-34.599757,7.0,FACULTAD DE MEDICINA,D


Creo una función para crear un punto con las coordenadas geográficas

In [ ]:
# función para convertir coordenadas x e y en un objeto punto.
from shapely.geometry import Point

def from_x_y(df, x, y):
    gdf = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'}, geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf

In [ ]:
caba_geo = from_x_y(data, 'lon', 'lat')
subtes_geo = from_x_y(subtes, 'long', 'lat')

Realizo una proyección para poder calcular las distancias

In [ ]:
caba_gkba = caba_geo.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
subtes_gkba = subtes_geo.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")

Defino una función que se queda con la distancia, el nombre y la línea de la estación más cercana

In [ ]:
def distancia_subte(x):
  distancias = []
  for y in range(len(subtes_gkba)):
    est = subtes_gkba.geometry[y]
    dist = x.distance(est)
    distancias.append(dist)
  estacion = np.argmin(distancias)
  return min(distancias), subtes_gkba.iloc[estacion]['estacion'], subtes_gkba.iloc[estacion]['linea']

Aplico la función al dataframe

In [ ]:
caba_gkba['dist_subte'], caba_gkba['estacion_subte_cercana'], caba_gkba['linea_subte_cercana'] = zip(*caba_gkba['geometry'].apply(lambda x: distancia_subte(x)))

In [ ]:
data = data.join(caba_gkba[['dist_subte', 'estacion_subte_cercana', 'linea_subte_cercana']])